<a href="https://colab.research.google.com/github/OVP2023/NLP/blob/main/API_deepseek_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain

In [ ]:
#Обработка текста, токенизация - деление на чанки
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt_tab')

text = "Считай по нашему мы выпили немного. Не вру ейбогу - скажи Серега. И если б водку бы не гнали из опилок чего б нам было с трех, четырех, пяти бутылок. Сначала пили близ прилавка в закуточки, но это были еще цветочки, но а когда коляска подкатила тогда у нас было по две на рыло."
chunks = sent_tokenize(text)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
chunks

['Считай по нашему мы выпили немного.',
 'Не вру ейбогу - скажи Серега.',
 'И если б водку бы не гнали из опилок чего б нам было с трех, четырех, пяти бутылок.',
 'Сначала пили близ прилавка в закуточки, но это были еще цветочки, но а когда коляска подкатила тогда у нас было по две на рыло.']

In [ ]:
#Генерация эмбеддингов через sentence_transformers

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")   #paraphrase-multilingual-MiniLM-L12-v2
embeddings = model.encode(chunks)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
embeddings

array([[ 0.02307638,  0.04903948, -0.01733485, ...,  0.08246319,
         0.02260365, -0.07418048],
       [ 0.01886017,  0.15764475, -0.05373492, ...,  0.05890214,
        -0.01029115, -0.0478964 ],
       [ 0.00717446,  0.06206214,  0.02017272, ...,  0.03750903,
         0.05552401, -0.10624763],
       [-0.00758227,  0.0512202 ,  0.00443223, ...,  0.02030315,
        -0.00122372, -0.05814464]], dtype=float32)

In [ ]:
embeddings.shape

(4, 384)

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 51.5 MB/s eta 0:00:00


In [ ]:
#Создание векторной базы данных FAISS  (Chroma не такая быстрая) https://habr.com/ru/companies/okkamgroup/articles/509204/

import faiss
import numpy as np

dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings).astype("float32"))
faiss.write_index(index, "my_index.faiss")

In [ ]:
print(index.ntotal) #количество индексов FAISS

4


In [ ]:
topn = 2
D, I = index.search(embeddings[:3], topn)  # найдем 2 ближайших соседей для первых 3 векторов. Возвращает результат: Distances, Indices
print(I)
print(D)

NameError: name 'index' is not defined

In [ ]:
index1 = faiss.read_index("my_index.faiss") #Прочитаем векторную DB с индексами
index1
D, I = index1.search(embeddings[:3], topn)  # найдем 2 ближайших соседей для первых 3 векторов. Возвращает результат: Distances, Indices
print(I)


[[0 1]
 [1 0]
 [2 3]]


КЛАСТЕРИЗАЦИЯ в FAISS с помощью алгоритма k-means

В этом и состоит идея IVF индекса: сгруппируем большой набор векторов по частям с помощью алгоритма k-means, каждой части поставив в соответствие центроиду, – вектор, являющийся выбранным центром для данного кластера. Поиск будем осуществлять через минимальное расстояние до центроид, и только потом искать минимальные расстояния среди векторов в том кластере, что соответствует данной центроиде. Взяв k равным $\sqrt{n}$, где $n$ – количество векторов в индексе, мы получим оптимальный поиск на двух уровнях – сначала среди $\sqrt{n}$ центроид, затем среди $\sqrt{n}$ векторов в каждом кластере. Поиск по сравнению с полным перебором ускоряется в разы, что решает одну из наших проблем при работе с множеством миллионов векторов.


Пространство векторов разбивается методом k-means на k кластеров. Каждому кластеру в соответствие ставится центроида

Пример кода:


dim = 512
k = 1000  # количество “командиров”

quantiser = faiss.IndexFlatL2(dim)

index = faiss.IndexIVFFlat(quantiser, dim, k)

vectors = np.random.random((1000000, dim)).astype('float32')  # 1 000 000 “воинов”



А можно это записать куда более элегантно, воспользовавшись удобной штукой FAISS для построения индекса:

index = faiss.index_factory(dim, “IVF1000,Flat”)

Запускаем обучение:

print(index.is_trained)   # False.

index.train(vectors)  # Train на нашем наборе векторов


Обучение завершено, но векторов в индексе пока нет, так что добавляем их в индекс:
print(index.is_trained)  # True

print(index.ntotal)   # 0

index.add(vectors)

print(index.ntotal)   # 1000000



Рассмотрев такой тип индекса после Flat, мы решили одну из наших потенциальных проблем – скорость поиска, которая становится в разы меньше по сравнению с полным перебором.

In [ ]:
#Теперь интегрируем LLM модель но не OpenAI c требуемым ключем, а с LLM моделzvb с Hugging Face. У Hugging Face и LangChain существует партнерский пакет, который предоставляет простой доступ к LLM

In [ ]:
!pip install langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from langchain_huggingface import HuggingFacePipeline,ChatHuggingFace,HuggingFaceEndpoint
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
import os

os.environ["HF_TOKEN"]='hf_ZNEyXStFKuveKbaVzcOYeQFOzlrmNmCH**'

In [ ]:
model_repo_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

llm = HuggingFacePipeline.from_model_id(
    model_id=model_repo_id,
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 100,
        "top_k": 50,
        "temperature": 0.1,
    }
)


'''#К модели обращаемся по IP, поэтому она может быть занята и попросит обратиться позднее
llm = HuggingFaceEndpoint(
    repo_id=model_repo_id,
    temperature=0.8,
    task="text-generation",
    max_new_tokens=1000,
    do_sample=False,
)'''


#ChatHuggingFace помогает составить правильный запрос к модели и является оберткой поверх llm
model = ChatHuggingFace(llm=llm)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Отвечай по русски."),
    ("user", "Расскажи о солнце")
])

#используем LCEL
chain = prompt | model

print(chain.invoke({}))